<a href="https://colab.research.google.com/github/waltechel/KaggleStudy20220103/blob/master/chapter04/stock_samsung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import
- 해당 라이브러리는 초기화한 다음에 런타임 초기화하면 안된다

In [6]:
!pip install -U finance-datareader

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
# from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
import datetime
import FinanceDataReader as fdr

# Load Dataset

- 이렇게 데이터를 쓸 수 있어요

```python
format = '%Y-%m-%d'
df_KOSPI = fdr.StockListing('KOSPI') # 코스피
df_KOSDAQ = fdr.StockListing('KOSDAQ') # 코스닥
df_KOSPI = pd.concat([df_KOSPI, df_KOSDAQ])
total_dataframe = pd.DataFrame()
```

- 이렇게 하면 데이터를 가져올 수 있어요.

In [32]:
# 005930 삼성전자
# BTC/KRW 비트코인
# ETH/KRW 이더리움
# XRP/KRW 리플
# BCH/KRW 비트코인 캐시
# EOS/KRW 이오스
# LTC/KRW 라이트 코인
# XLM/KRW 스텔라
df = fdr.DataReader('005930', '2016')

In [33]:
df.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2016-01-04,25200,25200,24100,24100,306939,-0.043651
2016-01-05,24040,24360,23720,24160,216002,0.002490
2016-01-06,24160,24160,23360,23500,366752,-0.027318
2016-01-07,23320,23660,23020,23260,282388,-0.010213
2016-01-08,23260,23720,23260,23420,257763,0.006879


# Compute Mid Price

In [34]:
high_prices = df['High'].values
low_prices = df['Low'].values
mid_prices = (high_prices + low_prices) / 2

# Create Windows

In [35]:
seq_len = 50
sequence_length = seq_len + 1

result = []
for index in range(len(mid_prices) - sequence_length):
    result.append(mid_prices[index: index + sequence_length])

# Normalize Data

- 이거는 완전히 사기 
  - 중간에 값을 채우는 기법밖에는 되지 않음
  - 10개 중에 9개 점 찍어 놓고 1개의 점을 채워넣는 기법
  - 미래를 예측하는 것이 아니다

In [36]:
normalized_data = []
for window in result:
    if float(window[0]) == 0:
        continue
    normalized_window = [((float(p) / float(window[0])) - 1) for p in window]
    normalized_data.append(normalized_window)

result = np.array(normalized_data)

# split train and test data
row = int(round(result.shape[0] * 0.9))
train = result[:row, :]
np.random.shuffle(train)

x_train = train[:, :-1]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = train[:, -1]

x_test = result[row:, :-1]
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
y_test = result[row:, -1]

x_train.shape, x_test.shape

((1299, 50, 1), (144, 50, 1))

# Build a Model

In [37]:
model = Sequential()

model.add(LSTM(50, return_sequences=True, input_shape=(50, 1)))

model.add(LSTM(64, return_sequences=False))

model.add(Dense(1, activation='linear'))

model.compile(loss='mse', optimizer='rmsprop')

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 50, 50)            10400     
                                                                 
 lstm_7 (LSTM)               (None, 64)                29440     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 39,905
Trainable params: 39,905
Non-trainable params: 0
_________________________________________________________________


# Training

In [ ]:
model.fit(x_train, y_train,
    validation_data=(x_test, y_test),
    batch_size=10,
    epochs=20)

Epoch 1/20
130/130 [==============================] - 9s 43ms/step - loss: 0.0049 - val_loss: 0.0018
Epoch 2/20
130/130 [==============================] - 5s 37ms/step - loss: 0.0039 - val_loss: 6.1253e-04
Epoch 3/20
130/130 [==============================] - 5s 38ms/step - loss: 0.0036 - val_loss: 5.0815e-04
Epoch 4/20
 53/130 [===========>..................] - ETA: 2s - loss: 0.0054

# Prediction

In [ ]:
pred = model.predict(x_test)

fig = plt.figure(facecolor='white', figsize=(20, 10))
ax = fig.add_subplot(111)
ax.plot(y_test, label='True')
ax.plot(pred, label='Prediction')
ax.legend()
plt.show()